In [1]:
import numpy as np
import torch
import torch.nn as nn
from muxcnn.models.ResNetHer import ResNetHer
from muxcnn.models.lightning import LitModel
from muxcnn.utils import load_params, load_img
from muxcnn.resnet_muxconvHer import ResNet_MuxConvHer
fn_chk = "./smooth_sep_bn3.pt"

/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/lightning_utilities/core/imports.py:132: Unbuilt egg for bbsQt [unknown version] (/home/hoseung/Work/FHE/Kinect_BBS_demo)
/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/lightning_utilities/core/imports.py:132: Unbuilt egg for bbsQt [unknown version] (/home/hoseung/Work/FHE/Kinect_BBS_demo)


# Comapre MuxedHer and TorchHer

## Torch

In [2]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']
model = ResNetHer([1,1,1])
model.eval()
model.to('cpu')

# Instantiate the model and Lightning module
criterion = nn.CrossEntropyLoss()

lit_model = LitModel.load_from_checkpoint(fn_chk, 
                                          model=model, 
                                          criterion=criterion,
                                          map_location=torch.device('cpu'))

In [3]:
img_tensor = load_img("./cute.jpg", hi=32, wi=32)

In [4]:
torch_result = lit_model(img_tensor)
print(torch_result)
torch_class = torch.argmax(torch_result)
print(f"[PyTorch] It's a {classes[torch_class]}")

tensor([[-14.2979, -18.1204,  -3.9596, -10.2655, -13.0855,  -0.0199, -13.6690,
          -7.4303, -15.6138, -18.0879]], grad_fn=<LogSoftmaxBackward0>)
[PyTorch] It's a dog


## Mux

In [5]:
muxed_model = ResNet_MuxConvHer(model)
img_tensor = load_img("./cute.jpg", hi=32, wi=32)

In [6]:
result = muxed_model(img_tensor)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
1 [-0.93062665]
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
2 [1.38483201]
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
1 [-0.65823687]
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 8
2 [1.78798806]
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =( 8, 8,64, 4, 4,  8)
[MultParConv] q = 16
1 [-0.58107

In [7]:
print(result[::64])
torch_class = np.argmax(result[::64][:10])
print(f"[Muxed] It's a {classes[torch_class]}")

[-7.31288361 -7.23378001 -6.57424678 ... -6.902275   -7.4838038
 -7.72304292]
[Muxed] It's a dog
